https://www.geeksforgeeks.org/microsoft-stock-price-prediction-with-machine-learning/



In [1]:
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns

In [4]:
import sys

sys.path.append(r'\Users\jimhu\option_trader\src')
from tradingBot.utils.optionTool import delta_call
from tradingBot.utils.optionTool import delta_put
from tradingBot.utils.optionTool import time_to_maturity_in_year
from tradingBot.utils.dataGetter import get_multiple_option_chains
from tradingBot.utils.dataGetter import get_option_exp_date
from tradingBot.utils.dataGetter import get_price_history

from datetime import time, date, datetime, timedelta
from pytz import timezone

from tradingBot.settings.tradeConfig import entryCrit
from tradingBot.settings.tradeConfig import runtimeConfig
from tradingBot.utils.dataGetter import get_price_history
from tradingBot.utils.dataGetter import get_option_exp_date
from tradingBot.consts import asset as at
from tradingBot.excel import positionSheet as ps


In [7]:
symbol = 'SPY'
period = '30y'
interval = '1d'

df = get_price_history(symbol, period=period, interval=interval)

df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
1993-05-25 00:00:00-04:00,25.923192,25.923192,25.869335,25.869335,191800,0.0,0.0,0.0
1993-05-26 00:00:00-04:00,25.941138,26.210423,25.923185,26.192471,102400,0.0,0.0,0.0
1993-05-27 00:00:00-04:00,26.228364,26.228364,26.066793,26.102697,53800,0.0,0.0,0.0
1993-05-28 00:00:00-04:00,26.084753,26.084753,25.851373,25.977039,79100,0.0,0.0,0.0
1993-06-01 00:00:00-04:00,26.066809,26.318142,26.030904,26.228380,28300,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2023-05-18 00:00:00-04:00,414.899994,419.670013,414.670013,419.230011,97177200,0.0,0.0,0.0
2023-05-19 00:00:00-04:00,420.170013,420.720001,417.350006,418.619995,103679700,0.0,0.0,0.0
2023-05-22 00:00:00-04:00,418.640015,420.390015,417.350006,418.790009,60745400,0.0,0.0,0.0


In [8]:
symbol = 'MSFT'

calls, puts = get_multiple_option_chains(symbol)

calls

,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,exp_date,bidSize,askSize,open,high,low,close,delta,gamma,vega,theta,sigma
0,150.0,130.35,0.0,0.0,2.0,0,0.000010,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1,220.0,59.40,0.0,0.0,2.0,0,0.000010,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,230.0,82.65,0.0,0.0,NaN,0,0.000010,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
3,240.0,75.00,0.0,0.0,2.0,0,0.000010,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,245.0,76.87,0.0,0.0,1.0,0,0.000010,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,415.0,0.15,0.0,0.0,39.0,0,0.125009,2023-08-18,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
58,420.0,0.14,0.0,0.0,13.0,0,0.125009,2023-08-18,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
59,435.0,0.07,0.0,0.0,60.0,0,0.125009,2023-08-18,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
60,440.0,0.03,0.0,0.0,NaN,0,0.125009,2023-08-18,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [9]:
current_price = df['Close'].tail(1)

In [10]:
calls['price'] = (calls['bid'] + calls['ask']) / 2
puts['price'] = (puts['bid'] + puts['ask']) / 2   

In [11]:
expected_price = float(current_price.values[0]) - 10

print('expected price', expected_price)

calls['gain'] = calls.apply(lambda x: -x['price'] if x['strike'] > expected_price else 100 * (expected_price-x['strike']-x['price'])/x['price'], axis = 1)

puts['gain'] = puts.apply(lambda x: -x['price'] if expected_price > x['strike']  else 100 * (x['strike'] - expected_price - x['price'])/x['price'], axis = 1)

expected price 401.0899963378906


ZeroDivisionError: float division by zero

In [12]:
calls

,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,exp_date,bidSize,askSize,open,high,low,close,delta,gamma,vega,theta,sigma,price
0,150.0,130.35,0.0,0.0,2.0,0,0.000010,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0
1,220.0,59.40,0.0,0.0,2.0,0,0.000010,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0
2,230.0,82.65,0.0,0.0,NaN,0,0.000010,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0
3,240.0,75.00,0.0,0.0,2.0,0,0.000010,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0
4,245.0,76.87,0.0,0.0,1.0,0,0.000010,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,415.0,0.15,0.0,0.0,39.0,0,0.125009,2023-08-18,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0
58,420.0,0.14,0.0,0.0,13.0,0,0.125009,2023-08-18,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0
59,435.0,0.07,0.0,0.0,60.0,0,0.125009,2023-08-18,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0
60,440.0,0.03,0.0,0.0,NaN,0,0.125009,2023-08-18,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0


In [ ]:
puts

In [ ]:
calls['lastTradeDate'].strftime("%Y-%m-%d")

In [ ]:
#calls['Exp'] = calls.apply(lambda x: x['lastTradeDate'].strftime("%Y-%m-%d"), axis = 1)
calls['Exp'] = calls.apply(lambda x: x['lastTradeDate'].date(), axis = 1)


In [ ]:
type(calls['Exp'][0])

In [ ]:
q = yf.Ticker(symbol)

q.option_chain()

In [2]:
import numpy as np
from math import sqrt, pi,log, e
from scipy.stats import norm
import time
import copy

class Option(object):
    def __init__(self, args):
        self.Type = int(args[0])                # 1 for a Call, - 1 for a put
        self.S = float(args[1])                 # Underlying asset price
        self.K = float(args[2])                 # Option strike K
        self.r = float(args[3])                 # Continuous risk fee rate
        self.q = float(args[4])                 # Dividend continuous rate
        self.T = float(args[5]) / 365.0         # Compute time to expiry
        self._sigma = float(args[6])             # Underlying volatility
        self.sigmaT = self._sigma * self.T ** 0.5# sigma*T for reusability

    @property
    def d2(self):
        return self.d1 - self.sigmaT

class BSVanilla(Option):
    def __init__(self, args):
        super(BSVanilla, self).__init__(args)

    @property
    def d1(self):
        return (log(self.S / self.K) + (self.r - self.q + 0.5 * (self.sigma ** 2)) * self.T) / self.sigmaT

    @property
    def sigma(self):
        return self._sigma
    @sigma.setter
    def sigma(self, val):
        self._sigma = val
        self.sigmaT = val * self.T ** 0.5

    def Premium(self):
        tmpprem = self.Type * (self.S * e ** (-self.q * self.T) * norm.cdf(self.Type * self.d1) - \
				self.K * e ** (-self.r * self.T) * norm.cdf(self.Type * self.d2))
        return tmpprem

    ############################################
    ############ 1st order greeks ##############
    ############################################

    def Delta(self):
        dfq = e ** (-self.q * self.T)
        if self.Type == 1:
            return dfq * norm.cdf(self.d1)
        else:
            return dfq * (norm.cdf(self.d1) - 1)

    # Vega for 1% change in vol
    def Vega(self):
        return 0.01 * self.S * e ** (-self.q * self.T) * \
          norm.pdf(self.d1) * self.T ** 0.5

    # Theta for 1 day change
    def Theta(self):
        df = e ** -(self.r * self.T)
        dfq = e ** (-self.q * self.T)
        tmptheta = (1.0 / 365.0) \
            * (-0.5 * self.S * dfq * norm.pdf(self.d1) * \
               self.sigma / (self.T ** 0.5) + \
            self.Type * (self.q * self.S * dfq * norm.cdf(self.Type * self.d1) \
            - self.r * self.K * df * norm.cdf(self.Type * self.d2)))
        return tmptheta

    def Rho(self):
        df = e ** -(self.r * self.T)
        return self.Type * self.K * self.T * df * 0.01 * norm.cdf(self.Type * self.d2)

    def Phi(self):
        return 0.01 * -self.Type * self.T * self.S * \
             e ** (-self.q * self.T) * norm.cdf(self.Type * self.d1)

    ############################################
    ############ 2nd order greeks ##############
    ############################################

    def Gamma(self):
        return e ** (-self.q * self.T) * norm.pdf(self.d1) / (self.S * self.sigmaT)

    # Charm for 1 day change
    def Charm(self):
        dfq = e ** (-self.q * self.T)
        if self.Type == 1:
            return (1.0 / 365.0) * -dfq * (norm.pdf(self.d1) * ((self.r - self.q) / (self.sigmaT) - self.d2 / (2 * self.T)) \
                            + (-self.q) * norm.cdf(self.d1))
        else:
            return (1.0 / 365.0) * -dfq * (norm.pdf(self.d1) * ((self.r - self.q) / (self.sigmaT) - self.d2 / (2 * self.T)) \
                            + self.q * norm.cdf(-self.d1))

    # Vanna for 1% change in vol
    def Vanna(self):
        return 0.01 * -e ** (-self.q * self.T) * self.d2 / self.sigma * norm.pdf(self.d1)

    # Vomma
    def Vomma(self):
        return 0.01 * -e ** (-self.q * self.T) * self.d2 / self.sigma * norm.pdf(self.d1)

# Geometric Continuous Average-Rate Options
# Following Kemna and Vorst (1990), see Haug
class BSAsian(Option):
    def __init__(self, args):
        super(BSAsian, self).__init__(args)
        self.sigmaT = self.sigmaa * self.T ** 0.5

    @property
    def d1(self):
        return (log(self.S / self.K) + (self.ba + 0.5 * (self.sigmaa ** 2)) * self.T) / self.sigmaT

    @property
    def sigmaa(self):
        return self._sigma / (3.0 ** 0.5)

    @property
    def sigma(self):
        return self._sigma
    @sigma.setter
    def sigma(self, val):
        self._sigma = val
        self.sigmaT = self.sigmaa * self.T ** 0.5

    @property
    def ba(self):
        return 0.5 * (self.r - self.q - (self._sigma ** 2) / 6.0)

    def Premium(self):
        tmpprem = self.Type * (self.S * e ** ((self.ba - self.r) * self.T) * norm.cdf(self.Type * self.d1) - \
                self.K * e ** (-self.r * self.T) * norm.cdf(self.Type * self.d2))
        return tmpprem

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
from matplotlib import cm
import OptionsAnalytics
from OptionsAnalytics import BSAsian

# Option parameters
sigma = 0.12        # Flat volatility
strike = 105.0      # Fixed strike
epsilon = 0.4       # The % on the left/right of Strike. 
                    # Asset prices are centered around Spot ("ATM Spot")
shortexpiry = 30    # Shortest expiry in days
longexpiry = 720    # Longest expiry in days
riskfree = 0.00     # Continuous risk free rate
divrate = 0.00      # Continuous div rate

# Grid definition
dx, dy = 40, 40     # Steps throughout asset price and expiries axis

# xx: Asset price axis, yy: expiry axis, zz: greek axis
xx, yy = np.meshgrid(np.linspace(strike*(1-epsilon), (1+epsilon)*strike, dx), \
    np.linspace(shortexpiry, longexpiry, dy))
print("Calculating greeks ...")
zz = np.array([BSAsian([1,x,strike,riskfree,divrate,y,sigma]).Premium() for
               x,y in zip(np.ravel(xx), np.ravel(yy))])
zz = zz.reshape(xx.shape)

# Plot greek surface
print("Plotting surface ...")
fig = plt.figure()
fig.suptitle('Call Charm',fontsize=20)
ax = fig.gca(projection='3d')
surf = ax.plot_surface(xx, yy, zz,rstride=1, cstride=1,alpha=0.75,cmap=cm.RdYlBu)
ax.set_xlabel('Asset price')
ax.set_ylabel('Expiry')
ax.set_zlabel('Charm')

# Plot 3D contour
zzlevels = np.linspace(zz.min(),zz.max(),num=8,endpoint=True)
xxlevels = np.linspace(xx.min(),xx.max(),num=8,endpoint=True)
yylevels = np.linspace(yy.min(),yy.max(),num=8,endpoint=True)
cset = ax.contourf(xx, yy, zz, zzlevels, zdir='z',offset=zz.min(),
                   cmap=cm.RdYlBu,linestyles='dashed')
cset = ax.contourf(xx, yy, zz, xxlevels, zdir='x',offset=xx.min(),
                   cmap=cm.RdYlBu,linestyles='dashed')
cset = ax.contourf(xx, yy, zz, yylevels, zdir='y',offset=yy.max(),
                   cmap=cm.RdYlBu,linestyles='dashed')

for c in cset.collections:
    c.set_dashes([(0, (2.0, 2.0))]) # Dash contours

plt.clabel(cset,fontsize=10, inline=1)

ax.set_xlim(xx.min(),xx.max())
ax.set_ylim(yy.min(),yy.max())
ax.set_zlim(zz.min(),zz.max())

#ax.relim()
#ax.autoscale_view(True,True,True)

# Colorbar
colbar = plt.colorbar(surf, shrink=1.0, extend='both', aspect = 10)
l,b,w,h = plt.gca().get_position().bounds
ll,bb,ww,hh = colbar.ax.get_position().bounds
colbar.ax.set_position([ll, b+0.1*h, ww, h*0.8])

# Show chart
plt.show()

ModuleNotFoundError: No module named 'OptionsAnalytics'

In [8]:
import numpy as np
from scipy.stats import norm
import sys

def binomialTree(price0, strike, vol, interest, time, dividend, putcall, binomial_steps, binomial_model, exercise):
    """

    :param price0: Current underlying asset price
    :param strike: strike price of the option
    :param vol: annualized volatility.
    :param interest: interest rate continuously compounded
    :param time: time in years (6 months == 6/12 == .5)
    :param dividend: annualized
    :param putcall: option type, either put or call
            'c' = call
            'p' = put
    :param binomial_steps: Only when using the binomial tree, the number of increments
    :param binomial_model:
            'base' - each step up is exp^(vol * sqrt(time/binomial_steps)) step down is exp^(-vol * sqrt(time/binomial_steps))
            'cox-ross-rubenstein' - in progress
            'jarrow-rudd' - in progress, the lognormal version of base
    :param exercise: the type of exercise for the option
            'a' - american, early exercise permitted
            'e' - european, exercise at expiration only
    :return: list with results
    """

    underlying = np.zeros((binomial_steps + 1, binomial_steps +1))
    option = np.zeros((binomial_steps + 1, binomial_steps +1))
    dt = time / binomial_steps

    sgn = 0
    if putcall == 'c':
        sgn = 1
    elif putcall == 'p':
        sgn = -1
    else:
        print("putcall variable is not properly defined, must be 'c' or 'p'")
        sys.exit()

    if binomial_model == 'base':
        u = np.exp((interest - dividend) * dt + vol * np.sqrt(dt))
        d = np.exp((interest - dividend) * dt - vol * np.sqrt(dt))
    elif binomial_model == 'cox-ross-rubenstein':
        u = np.exp(vol * np.sqrt(dt))
        d = 1/u
    elif binomial_model == 'jarrow-rudd':
        u = np.exp((interest-dividend-(vol**2)/2)*dt+vol*np.sqrt(dt))
        d = np.exp((interest-dividend-(vol**2)/2)*dt-vol*np.sqrt(dt))
    else:
        print("Value unrecognized in binomial_tree variable")
        sys.exit()

    for i in range(0,underlying.shape[0]):
        underlying[0,i] = price0 * u**(i)
    for i in range(1,underlying.shape[0]):
        underlying[i,:] = np.insert(underlying[i-1,:]*d, 0,0)[:-1]
    underlying = np.where(underlying < 0, 0, underlying)

    a = np.exp((interest-dividend)*dt)
    pu = (a-d)/(u-d)
    pd0 = 1 - pu
    pv0 = np.exp( -interest * dt)


    if putcall == 'c':
        option[:,binomial_steps] = underlying[:,(binomial_steps)] - strike
        option = np.where(option < 0, 0, option)
        if exercise == 'e':
            for i in reversed(range(binomial_steps + 1)):
                option[0:i,(i-1)] = pv0 * (pu * option[0:i,i] + pd0 * option[1:(i+1),i])
        else:
            for i in reversed(range(binomial_steps + 1)):
                tval = pv0 * (pu * option[0:i,i] + pd0 * option[1:(i+1),i])
                ex = (underlying[0:i,i] - strike)
                option[0:i,(i-1)] = np.maximum(tval, ex)
    elif putcall == 'p':
        option[:,binomial_steps] = strike - underlying[:,(binomial_steps)]
        option = np.where(option < 0, 0, option)
        for i in reversed(range(binomial_steps + 1)):
            option[0:i,(i-1)] = pv0 * (pu * option[0:i,i] + pd0 * option[1:(i+1),i])
        else:
            for i in reversed(range(binomial_steps + 1)):
                tval = pv0 * (pu * option[0:i,i] + pd0 * option[1:(i+1),i])
                ex = strike - underlying[0:i,i]
                option[0:i,(i-1)] = np.maximum(tval, ex)

    return round(float(option[0, 0]), 3)

def blackScholes(price0, 
                 strike, 
                 vol, 
                 interest, 
                 time, 
                 dividend, 
                 putcall):
    d1 = (np.log(price0/ strike) + (interest - dividend + (vol**2)/2) * time)/ (vol * np.sqrt(time))
    print('d1=', d1)
    d2 = d1 - vol * np.sqrt(time)
    pn1 = norm.cdf(d1)
    pn2 = norm.cdf(d2)
    pn3 = norm.cdf(-d2)
    pn4 = norm.cdf(-d1)
    def n1(x):
        return np.exp(-(x**2)/ 2)/ np.sqrt(2* np.pi)

    if putcall == 'c':
        optionPrice = price0 * np.exp(-dividend * time) * pn1 - strike * np.exp(-interest * time) * pn2
        optionDelta = pn1 * np.exp(-dividend * time)
        optionGamma = n1(d1) * np.exp(-dividend*time)/ (price0 * vol * np.sqrt(time))
        optionTheta = dividend* price0* pn1* np.exp(-dividend* time) - interest* strike* np.exp(-interest* time)* pn2 - price0* n1(d1)* vol* np.exp(-dividend* time)/(2* np.sqrt(time))
        optionVega = price0* n1(d1)* np.sqrt(time)* np.exp(-dividend * time)/ 100
    elif putcall == 'p':
        optionPrice = strike * np.exp(-interest * time) * pn3 - price0 * np.exp(-dividend * time) * pn4
        optionDelta = (pn1 - 1) * np.exp(-dividend * time)
        optionGamma = n1(d1) * np.exp(-dividend*time)/ (price0 * vol * np.sqrt(time))
        optionTheta = -dividend* price0* pn4* np.exp(-dividend* time) + interest* strike* np.exp(-interest* time)* pn3 - price0* n1(d1)* vol* np.exp(-dividend* time)/(2* np.sqrt(time))
        optionVega = price0* n1(d1)* np.sqrt(time)* np.exp(-dividend * time)/ 100
    else:
        print("Incorrect value in the putcall var, must be a 'p' or 'c'")

    return optionPrice, optionDelta, optionGamma, optionTheta, optionVega


#testing the output
if __name__ == '__main__':
    print(binomialTree(100, 90, .25, 0.01, 2/12, 0, 'p', 30, 'base', 'e'))
    print(blackScholes(100, 90, .25, 0.01, 2/12, 0, 'p'))

0.726
d1= 1.0996789775193057
(0.7334663562810579, -0.13573600874089808, 0.021352604625075836, -6.52961827303249, 0.08896918593781597)


In [ ]:
S=

def d1(S, K, T, r, sigma):
    if T <= 0:
        logger.error('Zero T')      
    return (np.log(S/K) + (r + sigma**2/2)*T) / sigma*np.sqrt(T)
    
def d2(S, K, T, r, sigma):
    return d1(S, K, T, r, sigma) - sigma* np.sqrt(T)

def delta_call(S, K, T, sigma=0.25, r=0.05):
    N = norm.cdf
    return N(d1(S, K, T, r, sigma))